In [49]:
import yfinance
import yfinance as yf

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

import os
import pickle

In [69]:
# 取得 AAPL 價量資料
df = yf.Ticker('2454.TW').history(start="2011-01-01", end="2021-07-08")
df.head()

# df = df.rename(columns={'Open': 'open','High': 'high','Low': 'low','Close': 'close','Volume': 'volume'})
df = df[df.columns[:5]] 
df.head(10)

,Open,High,Low,Close,Volume
Date,,,,,
2011-01-03,278.300927,278.961190,273.348953,274.009216,4683000
2011-01-04,277.970749,279.621407,275.989960,275.989960,6853000
2011-01-05,278.300911,279.951569,275.989990,277.310516,8747000
2011-01-06,277.970796,278.300927,269.057243,271.368164,7931000
2011-01-07,270.047624,270.047624,254.201309,257.502625,21146000
2011-01-10,254.201309,254.531441,249.579467,254.201309,6816000
2011-01-11,257.502624,264.765518,256.842361,262.784729,8594000
2011-01-12,269.387374,274.669479,268.066848,274.009216,12198000
2011-01-13,274.009217,275.659875,271.038032,271.368164,5819000


In [54]:
def crossover1(series1, series2) -> bool:
    """
    Return `True` if `series1` just crossed over
    `series2`.

        >>> crossover(self.data.Close, self.sma)
        True
    """
    import pandas as pd
    from numbers import Number
    series1 = (
        series1.values if isinstance(series1, pd.Series) else
        (series1, series1) if isinstance(series1, Number) else
        series1)
    series2 = (
        series2.values if isinstance(series2, pd.Series) else
        (series2, series2) if isinstance(series2, Number) else
        series2)
    try:
#         print(series1)
        return series1[-2] < series2[-2] and series1[-1] > series2[-1]
    except IndexError:
        return False

In [71]:
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 5) # 計算5日均綫
        self.ma2 = self.I(SMA, price, 20) #計算20日均綫 

    def next(self):
#         crossover1(self.ma1, self.ma2)
#         if os.path.isfile("./timeshow.pickle"):
#             if (self.ma1[-1]/self.ma2[-1]) >1:
#                 self.buy()
#                 os.remove('./timeshow.pickle')
        if crossover(self.ma1, self.ma2): # 如果ma1突破ma2
            self.buy()
#             with open('timeshow.pickle', 'wb') as f:
#                 timeshow = '1'
#                 pickle.dump(timeshow, f)
        elif crossover(self.ma2, self.ma1): # 如果ma2突破ma1
            self.sell()
#             if len(self.trades) > 0: # 如果此時有單在倉内 强制平倉
#                 self.trades[0].close()


bt = Backtest(df, SmaCross, commission=.006, cash = 10000, exclusive_orders=True)# commision 手續費， cash資金， exlusive order一次一筆交易
stats = bt.run()
bt.plot()

In [72]:
stats

Start                     2011-01-03 00:00:00
End                       2021-07-07 00:00:00
Duration                   3838 days 00:00:00
Exposure Time [%]                   98.327499
Equity Final [$]                  2448.709389
Equity Peak [$]                       10000.0
Return [%]                         -75.512906
Buy & Hold Return [%]              234.660276
Return (Ann.) [%]                  -12.882407
Volatility (Ann.) [%]               29.205528
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -87.435589
Avg. Drawdown [%]                  -87.435589
Max. Drawdown Duration     3771 days 00:00:00
Avg. Drawdown Duration     3771 days 00:00:00
# Trades                                  167
Win Rate [%]                        29.341317
Best Trade [%]                      80.500155
Worst Trade [%]                    -22.560751
Avg. Trade [%]                    